# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape       # Não há linhas duplicadas

(27028, 69)


(27028, 69)

In [2]:
# 2) seu código aqui
sinasc.isna().sum().sort_values(ascending=False)

DTRECORIGA    27028
CODANOMAL     26814
IDADEPAI      19421
SERIESCMAE    12710
DTULTMENST    10072
              ...  
PESO              0
DIFDATA           0
DTCADASTRO        0
DTRECEBIM         0
ORIGEM            0
Length: 69, dtype: int64

In [3]:
# 3) seu código aqui
colunas = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc = sinasc[colunas]
sinasc.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [4]:
# 4) seu código aqui
sinasc.dropna(subset=['APGAR5'], inplace=True)
print(sinasc.shape)
sinasc.isna().sum()

(26925, 9)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [5]:
# 5) seu código aqui
sinasc.loc[sinasc['ESTCIVMAE']==9,['ESTCIVMAE']] = None
sinasc.loc[sinasc['CONSULTAS']==9,['CONSULTAS']] = None

In [6]:
# 6) Seu código aqui
sinasc['QTDFILVIVO'].fillna(0, inplace=True)

In [7]:
# 7) seu código aqui
print(sinasc.isna().sum())

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      459
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS       50
APGAR5           0
dtype: int64


Vamos analisar as colunas 'ESCMAE', 'GESTACAO' e 'GRAVIDEZ', para revisar seus dados faltantes.

In [8]:
sinasc['ESCMAE'].value_counts(dropna=False)

8 a 11 anos        15581
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
NaN                  310
Nenhuma               40
Name: ESCMAE, dtype: int64

No caso da escolaridade da mãe, os valores faltantes podem ser considerados como uma nova categoria de 'sem informação', então não serão tratados de outra forma.

In [9]:
sinasc['GESTACAO'].value_counts(dropna=False)

37 a 41 semanas        22523
32 a 36 semanas         2234
NaN                     1216
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: GESTACAO, dtype: int64

In [10]:
sinasc['GRAVIDEZ'].value_counts(dropna=False)

Única              26356
Dupla                488
NaN                   76
Tríplice e mais        5
Name: GRAVIDEZ, dtype: int64

No caso do atributo de tempo de gestação e no tipo de gravidez, os valores faltantes realmente são um buraco nos dados pois têm muito valor para a análise, assim, tais linhas serão excluidas.

In [11]:
sinasc.dropna(subset=['GESTACAO', 'GRAVIDEZ'], inplace=True)

- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.

In [12]:
# 8) seu código aqui
sinasc['APGAR5'].value_counts(dropna=False)

9.0     15339
10.0     7976
8.0      1900
7.0       233
6.0        72
5.0        40
1.0        22
4.0        21
2.0        16
0.0        16
3.0        14
Name: APGAR5, dtype: int64

In [13]:
sinasc.loc[(sinasc['APGAR5'] >= 8) & (sinasc['APGAR5'] <= 10), 'APGAR5_cat'] = 'Normal'
sinasc.loc[(sinasc['APGAR5'] >= 6) & (sinasc['APGAR5'] <= 7), 'APGAR5_cat'] = 'Asfixia Leve'
sinasc.loc[(sinasc['APGAR5'] >= 4) & (sinasc['APGAR5'] <= 5), 'APGAR5_cat'] = 'Asfixia moderada'
sinasc.loc[(sinasc['APGAR5'] >= 0) & (sinasc['APGAR5'] <= 3), 'APGAR5_cat'] = 'Asfixia severa'

sinasc['APGAR5_cat'].value_counts(dropna=False)

Normal              25215
Asfixia Leve          305
Asfixia severa         68
Asfixia moderada       61
Name: APGAR5_cat, dtype: int64

In [14]:
# 9) seu código aqui
sinasc.drop('APGAR5', axis=1, inplace=True)
novos_nome = {'LOCNASC': 'local_nascimento',
              'IDADEMAE': 'idade_mae', 
              'ESTCIVMAE': 'estado_civil_mae',
              'ESCMAE': 'escolaridade_mae',
              'QTDFILVIVO': 'qtd_filho_vivo',
              'GESTACAO': 'tempo_gestacao',
              'GRAVIDEZ': 'tipo_gravidez',
              'CONSULTAS':'qtd_consultas',
              'APGAR5_cat': 'apgar_5'}
sinasc.rename(columns=novos_nome, inplace=True)

In [15]:
sinasc.head()

,local_nascimento,idade_mae,estado_civil_mae,escolaridade_mae,qtd_filho_vivo,tempo_gestacao,tipo_gravidez,qtd_consultas,apgar_5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,Normal
2,1,37,NaN,8 a 11 anos,2.0,37 a 41 semanas,Única,4.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,Normal
